# Comparación de los algoritmos implementados desde 0 con los algoritmos tomados de bibliotecas. 

### Bibliotecas

In [1]:
# Bibliotecas públicas
import random
import os
from sklearn import datasets
from operator import itemgetter
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans

# Bibliotecas personalizadas
from Scripts.dataset import open_dataset
from Scripts.knn import classify as knn_classify
from Scripts.evaluation_metrics import *
from Scripts.distance_metrics import *
from Scripts.confusion_matrix import confusion_matrix
from Scripts.dbscan import *

## Cargando los datos

In [2]:
# Se especifican las rutas al conjunto de entrenamiento y prueba
dataset1_train = open_dataset("Datasets/Datos-S-Entrena.txt", True)
dataset1_test = open_dataset("Datasets/S-Prueba.txt", True)

# Conjunto de datos unido (para validación cruzada)
dataset1_completo = dataset1_train + dataset1_test

# Aleatorizando el orden del conjunto de datos
random.shuffle(dataset1_completo)

***************************************************
Cantidad de ejemplos en el conjunto:  397
Cantidad de atributos: 18
Ejemplo de instancia:  ([140.0, 125.0, 0.0, 0.0, 0.0, 0.2777779, 0.06296301, 0.66666675, 0.31111118, 6.185185, 7.3333335, 7.6666665, 3.5555556, 3.4444444, 4.4444447, -7.888889, 7.7777777, 0.5456349], 0)
***************************************************
Cantidad de ejemplos en el conjunto:  23
Cantidad de atributos: 18
Ejemplo de instancia:  ([242.0, 65.0, 0.0, 0.0, 0.0, 0.5555553, 0.27216578, 1.0, 0.42163682, 8.925926, 6.111111, 14.333333, 6.3333335, -8.444445, 16.222221, -7.7777777, 14.333333, 0.5834605], 6)


## Realizando CLASIFICACIÓN con conjuntos de entrenamiento y prueba

### k vecinos más cercanos (implementado desde 0)

In [7]:
# Prediciendo sobre el conjunto de prueba
knn_predictions = knn_classify(dataset1_train, dataset1_test, k=3, metric='euclidean')

# Imprimiendo exactitud
print("Exactitud del clasificador: ", str(accuracy(knn_predictions, False)*100)+"%")

Exactitud del clasificador:  91.30434782608695%


In [9]:
# Imprimiento la matriz de confusión
confusion_matrix(dataset1_completo, knn_predictions)

Predicción,0,1,2,3,4,5,6
Real,,,,,,,
0,3,0,0,0,0,0,0
1,0,3,1,0,0,0,0
2,0,0,3,0,0,0,0
3,0,0,0,3,0,0,0
4,0,0,0,0,4,0,0
5,0,0,0,0,0,3,0
6,1,0,0,0,0,0,2


### Naive Bayes (tomado de la biblioteca sklearn)

In [10]:
# Se utilizará la versión que asume una distribución Gaussiana
def sk_naive_bayes(train_data, test_data):
    # Instanciando el método
    gnb = GaussianNB()

    # Formateando el conjunto de datos
    X_train = [sample[0] for sample in train_data]
    y_train = [sample[1] for sample in train_data]
    X_test = [sample[0] for sample in test_data]
    y_test = [sample[1] for sample in test_data]

    y_pred = gnb.fit(X_train, y_train).predict(X_test)

    # Formateando para utilizar nuestra función de matriz de confusión
    predictions = []
    for i in range(len(y_test)):
        predictions.append((y_test[i], y_pred[i]))
    
    return predictions

nb_predictions =sk_naive_bayes(dataset1_train, 
                               dataset1_test)

In [11]:
# Imprimiendo exactitud
print("Exactitud del clasificador: ", str(accuracy(nb_predictions, False)*100)+"%")

Exactitud del clasificador:  82.6086956521739%


In [12]:
# Imprimiento la matriz de confusión
confusion_matrix(dataset1_completo, nb_predictions)

Predicción,0,1,2,3,4,5,6
Real,,,,,,,
0,4,0,0,0,0,0,0
1,0,3,0,0,0,0,0
2,0,1,0,0,0,0,3
3,0,0,0,3,0,0,0
4,0,0,0,0,4,0,0
5,0,0,0,0,0,3,0
6,0,0,0,0,0,0,2


## Realizando CLASIFICACIÓN con validación cruzada

### k vecinos más cercanos (implementado desde 0)

In [14]:
# Realizando validación cruzada
# k_fold_cross_val(data, n_folds, k_vecinos, metric)
k_fold_predictions = k_fold_cross_val(dataset1_completo, 10, 2, 'euclidean')

# Ordenando por puntajes
k_fold_predictions.sort(key=itemgetter(1), reverse=True)

# Obteniendo puntajes y mostrando información
k_fold_scores = [score[1] for score in k_fold_predictions]
print("Puntajes por fold:", k_fold_scores)
print("Promedio de precisión en k-folds:", sum(k_fold_scores)/len(k_fold_scores))

Diviendo los datos en 10 folds de tamaño 42
Puntajes por fold: [0.9761904761904762, 0.9761904761904762, 0.9523809523809523, 0.9523809523809523, 0.9285714285714286, 0.9285714285714286, 0.9047619047619048, 0.9047619047619048, 0.8809523809523809, 0.8333333333333334]
Promedio de precisión en k-folds: 0.9238095238095239


In [15]:
# Imprimiento la matriz de confusión (mejor resultado)
best_prediction = k_fold_predictions[0][0]
print("Matriz de confusión del mejor resultado en los k-folds. \nExactitud:", k_fold_predictions[0][1])
confusion_matrix(dataset1_completo, best_prediction)

Matriz de confusión del mejor resultado en los k-folds. 
Exactitud: 0.9761904761904762


Predicción,0,1,2,3,4,5,6
Real,,,,,,,
0,2,0,0,0,0,0,1
1,0,3,0,0,0,0,0
2,0,0,3,0,0,0,0
3,0,0,0,7,0,0,0
4,0,0,0,0,11,0,0
5,0,0,0,0,0,4,0
6,0,0,0,0,0,0,11


In [16]:
# Imprimiento la matriz de confusión (mejor resultado)
worst_prediction = k_fold_predictions[-1][0]
print("Matriz de confusión del mejor resultado en los k-folds. \nExactitud:", k_fold_predictions[-1][1])
confusion_matrix(dataset1_completo, worst_prediction)

Matriz de confusión del mejor resultado en los k-folds. 
Exactitud: 0.8333333333333334


Predicción,0,1,2,3,4,5,6
Real,,,,,,,
0,2,0,1,0,0,0,0
1,0,4,0,0,0,0,0
2,1,0,0,0,0,0,1
3,0,0,0,8,0,0,0
4,0,1,0,0,11,0,1
5,0,0,0,0,0,2,0
6,0,1,1,0,0,0,8


### Naive Bayes (tomado de la biblioteca sklearn)

In [17]:
# Realizando validación cruzada
# k_fold_cross_val(data, n_folds, k_vecinos, metric)
k_fold_predictions = nb_k_fold_cross_val(dataset1_completo, 10, sk_naive_bayes)

# Ordenando por puntajes
k_fold_predictions.sort(key=itemgetter(1), reverse=True)

# Obteniendo puntajes y mostrando información
k_fold_scores = [score[1] for score in k_fold_predictions]
print("Puntajes por fold:", k_fold_scores)
print("Promedio de precisión en k-folds:", sum(k_fold_scores)/len(k_fold_scores))

Diviendo los datos en 10 folds de tamaño 42
Puntajes por fold: [0.9523809523809523, 0.9047619047619048, 0.8809523809523809, 0.8809523809523809, 0.8571428571428571, 0.8571428571428571, 0.8333333333333334, 0.8095238095238095, 0.8095238095238095, 0.7142857142857143]
Promedio de precisión en k-folds: 0.8499999999999999


In [18]:
# Imprimiento la matriz de confusión (mejor resultado)
best_prediction = k_fold_predictions[0][0]
print("Matriz de confusión del mejor resultado en los k-folds. \nExactitud:", k_fold_predictions[0][1])
confusion_matrix(dataset1_completo, best_prediction)

Matriz de confusión del mejor resultado en los k-folds. 
Exactitud: 0.9523809523809523


Predicción,0,1,2,3,4,5,6
Real,,,,,,,
0,6,0,0,0,0,0,0
1,0,3,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,5,0,0,0
4,0,0,0,0,9,0,0
5,0,0,0,0,0,4,0
6,1,0,1,0,0,0,13


In [19]:
# Imprimiento la matriz de confusión (mejor resultado)
worst_prediction = k_fold_predictions[-1][0]
print("Matriz de confusión del mejor resultado en los k-folds. \nExactitud:", k_fold_predictions[-1][1])
confusion_matrix(dataset1_completo, worst_prediction)

Matriz de confusión del mejor resultado en los k-folds. 
Exactitud: 0.7142857142857143


Predicción,0,1,2,3,4,5,6
Real,,,,,,,
0,3,0,0,0,0,0,1
1,2,2,1,0,0,0,1
2,1,0,0,0,0,0,2
3,0,0,0,6,0,0,2
4,0,2,0,0,7,0,0
5,0,0,0,0,0,2,0
6,0,0,0,0,0,0,10


## Realizando AGRUPAMIENTO

### DBSCAN (implementado desde 0)

In [25]:
# Añadiendo etiquetas vacías al dataset
DB_clase = add_labels(dataset1_test)

# Realizando agrupamiento con DBSCAN
clustered_db = DBSCAN(DB_clase, euclidean, 80, 2)
clusters = get_clusters(clustered_db, True)

Cluster 0 : [1, 16] 

Cluster 1 : [2, 3, 4, 9] 

Cluster 2 : [5, 6, 8, 13, 14, 15] 

Cluster 3 : [10, 11, 12] 

Cluster 4 : [17, 18] 

Cluster 5 : [20, 21, 22] 

Ruido: [0, 7, 19] 



### K-Means (tomado de la biblioteca sklearn)

In [26]:
# Formateando información
X = [sample[0] for sample in dataset1_test]

# Instanciando el agrupador
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)


# Formateando las salidas
clustered_db2 = []
for i, label in enumerate(kmeans.labels_):
    clustered_db2.append([X[i], label])

# Visualizando etiquetas
clusters = get_clusters(clustered_db2, True)

Cluster 0 : [17, 18, 19] 

Cluster 1 : [5, 6, 8, 13, 14, 15] 

Cluster 2 : [20, 21, 22] 

Cluster 3 : [10, 11, 12] 

Cluster 4 : [2, 3, 4, 9] 

Cluster 5 : [7] 

Cluster 6 : [0, 1, 16] 

